<a href="https://colab.research.google.com/github/felipe-lazzaron/OptiRoute-Efficient-Vehicle-Routing-with-Parallel-Computing/blob/main/GPU_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Tue Jun  4 03:29:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!apt-get update
!apt-get install -y cuda


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,085 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,129 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/univers

In [7]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Apr_17_19:19:55_PDT_2024
Cuda compilation tools, release 12.5, V12.5.40
Build cuda_12.5.r12.5/compiler.34177558_0


In [38]:
%%writefile parallel_vrp_thrust.cu

#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/sort.h>
#include <thrust/execution_policy.h>
#include <thrust/transform_reduce.h>
#include <iostream>
#include <vector>
#include <cmath>
#include <climits>
#include <chrono>
#include <numeric>
#include <cuda_runtime.h>
#include <cfloat>  // Inclui DBL_MAX

using namespace std;

#define CUDA_CHECK_ERROR() {                                           \
    cudaError_t err = cudaGetLastError();                              \
    if (err != cudaSuccess) {                                          \
        std::cerr << "CUDA Error: " << cudaGetErrorString(err) << std::endl;  \
        exit(err);                                                     \
    }                                                                  \
}

__host__ __device__
double calcularCusto(int* rota, double* distancias, int num_locais) {
    double custo = distancias[0 * num_locais + rota[0]];
    for (int i = 0; i < num_locais - 1; ++i) {
        custo += distancias[rota[i] * num_locais + rota[i + 1]];
    }
    custo += distancias[rota[num_locais - 1] * num_locais + 0];
    return custo;
}

__host__ __device__
bool verificarCapacidade(int* rota, int* demandas, int capacidade, int num_locais) {
    int cargaTotal = 0;
    for (int i = 0; i < num_locais; ++i) {
        cargaTotal += demandas[rota[i]];
        if (cargaTotal > capacidade) return false;
    }
    return true;
}

__global__
void processPermutations(int* permutas, double* distancias, int* demandas, int num_locais, int capacidade, double* min_costs, int* best_routes) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    extern __shared__ double shared_mem[];
    int* rota = (int*)shared_mem;
    double* min_cost = shared_mem + num_locais;

    if (idx < num_locais) {
        for (int i = 0; i < num_locais; ++i) {
            rota[i] = permutas[idx * num_locais + i];
        }

        if (verificarCapacidade(rota, demandas, capacidade, num_locais)) {
            double custo = calcularCusto(rota, distancias, num_locais);
            if (custo < *min_cost) {
                *min_cost = custo;
                for (int i = 0; i < num_locais; ++i) {
                    best_routes[idx * num_locais + i] = rota[i];
                }
            }
            min_costs[idx] = *min_cost;
        }
    }
}

int main() {
    vector<int> num_nodes = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12};
    vector<double> times;

    for (int n : num_nodes) {
        cout << "Calculando para " << n << " locais..." << endl;

        thrust::host_vector<int> h_locais(n);
        iota(h_locais.begin(), h_locais.end(), 1);

        thrust::device_vector<int> d_locais = h_locais;
        thrust::host_vector<double> h_distancias((n + 2) * (n + 2));
        thrust::host_vector<int> h_demandas(n + 2, 1);
        h_demandas[0] = 0;
        h_demandas[n + 1] = 0;

        for (int i = 1; i <= n; ++i) {
            h_distancias[0 * (n + 2) + i] = i;
            h_distancias[i * (n + 2) + 0] = i;
            h_distancias[i * (n + 2) + (n + 1)] = i;
            h_distancias[(n + 1) * (n + 2) + i] = i;
        }

        for (int i = 1; i < n; ++i) {
            for (int j = i + 1; j <= n; ++j) {
                h_distancias[i * (n + 2) + j] = abs(i - j);
                h_distancias[j * (n + 2) + i] = abs(i - j);
            }
        }

        thrust::device_vector<double> d_distancias = h_distancias;
        thrust::device_vector<int> d_demandas = h_demandas;
        int capacidade = 10;
        int num_permutations = tgamma(n + 1);
        int block_size = 256;
        int num_blocks = (num_permutations + block_size - 1) / block_size;

        thrust::device_vector<int> d_permutas(num_permutations * n);
        thrust::device_vector<double> d_min_costs(num_permutations, DBL_MAX);
        thrust::device_vector<int> d_best_routes(num_permutations * n);

        auto start = chrono::high_resolution_clock::now();

        size_t shared_memory_size = sizeof(int) * n + sizeof(double);
        processPermutations<<<num_blocks, block_size, shared_memory_size>>>(thrust::raw_pointer_cast(d_permutas.data()),
                                                                           thrust::raw_pointer_cast(d_distancias.data()),
                                                                           thrust::raw_pointer_cast(d_demandas.data()),
                                                                           n,
                                                                           capacidade,
                                                                           thrust::raw_pointer_cast(d_min_costs.data()),
                                                                           thrust::raw_pointer_cast(d_best_routes.data()));

        CUDA_CHECK_ERROR();
        cudaDeviceSynchronize();
        CUDA_CHECK_ERROR();

        auto end = chrono::high_resolution_clock::now();
        chrono::duration<double> duration = end - start;
        times.push_back(duration.count());

        cout << "Tempo para " << n << " locais: " << duration.count() << " segundos" << endl;
    }

    for (size_t i = 0; i < num_nodes.size(); ++i) {
        cout << "Locais: " << num_nodes[i] << " Tempo: " << times[i] << " segundos" << endl;
    }

    return 0;
}


Overwriting parallel_vrp_thrust.cu


In [39]:
!nvcc -arch=sm_75 -o parallel_vrp_thrust parallel_vrp_thrust.cu


In [40]:
!./parallel_vrp_thrust


Calculando para 1 locais...
Tempo para 1 locais: 2.8667e-05 segundos
Calculando para 2 locais...
Tempo para 2 locais: 1.2269e-05 segundos
Calculando para 3 locais...
Tempo para 3 locais: 1.2891e-05 segundos
Calculando para 4 locais...
Tempo para 4 locais: 1.2191e-05 segundos
Calculando para 5 locais...
Tempo para 5 locais: 1.328e-05 segundos
Calculando para 6 locais...
Tempo para 6 locais: 1.3536e-05 segundos
Calculando para 7 locais...
Tempo para 7 locais: 1.4201e-05 segundos
Calculando para 8 locais...
Tempo para 8 locais: 1.4157e-05 segundos
Calculando para 9 locais...
Tempo para 9 locais: 1.5958e-05 segundos
Calculando para 10 locais...
Tempo para 10 locais: 7.9141e-05 segundos
Calculando para 11 locais...
Tempo para 11 locais: 0.000441361 segundos
Calculando para 12 locais...
Tempo para 12 locais: 0.00514382 segundos
Locais: 1 Tempo: 2.8667e-05 segundos
Locais: 2 Tempo: 1.2269e-05 segundos
Locais: 3 Tempo: 1.2891e-05 segundos
Locais: 4 Tempo: 1.2191e-05 segundos
Locais: 5 Tempo: 1

In [15]:
!nvidia-smi


Tue Jun  4 03:42:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--